In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
from helper_functions import *
import pandas as pd
importTensorflow(memory=4090)

2.13.0
1 Physical GPUs, 1 Logical GPUs


In [2]:
df = pd.read_csv("queries.csv")
df.head()

,String,Class
0,Rich and buttery delicacy recipes,foie_gras
1,Luxurious appetizer with duck liver,foie_gras
2,Indulgent gourmet dish preparation,foie_gras
3,Recipes for savory liver pâté,foie_gras
4,Delicate and decadent French cuisine ideas,foie_gras


In [3]:
d = set()
counter = -1
l = []
for thing in df.Class:
    if thing in d:
      l.append(counter)
    else:
        counter += 1
        d.add(thing)
        l.append(counter)

In [4]:
class_list = list(set(df.Class))

In [5]:
df['classEncode'] = l

In [6]:
df.head()

,String,Class,classEncode
0,Rich and buttery delicacy recipes,foie_gras,0
1,Luxurious appetizer with duck liver,foie_gras,0
2,Indulgent gourmet dish preparation,foie_gras,0
3,Recipes for savory liver pâté,foie_gras,0
4,Delicate and decadent French cuisine ideas,foie_gras,0


In [7]:
df = df.sample(frac = 1)
df.head()

,String,Class,classEncode
2309,Meals featuring homemade classic pho,pho,93
1773,Cooking tutorials for searing pork chops to pe...,pork_chop,71
2399,Homemade dessert with a luxurious and indulgen...,creme_brulee,97
723,Savory starter with fresh ingredients and zest...,beef_tartare,29
2130,Recipes for lasagna with a sprinkle of Parmesa...,lasagna,86


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.String, df.classEncode, test_size=0.2)

In [9]:
len(X_train), len(X_test)

(1984, 496)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64).prefetch(tf.data.AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [11]:
vocab_length = 25000
max_length = [len(i.split()) for i in df.String]
max_length

[5,
 8,
 8,
 8,
 11,
 4,
 11,
 8,
 13,
 7,
 7,
 9,
 7,
 9,
 8,
 12,
 9,
 8,
 7,
 8,
 7,
 7,
 12,
 9,
 8,
 9,
 12,
 9,
 11,
 6,
 7,
 11,
 9,
 10,
 5,
 8,
 7,
 8,
 7,
 6,
 5,
 13,
 9,
 9,
 8,
 8,
 10,
 9,
 8,
 7,
 7,
 11,
 9,
 9,
 11,
 9,
 8,
 10,
 6,
 8,
 9,
 8,
 9,
 9,
 7,
 8,
 8,
 12,
 7,
 9,
 8,
 9,
 8,
 9,
 13,
 11,
 8,
 6,
 7,
 12,
 10,
 7,
 6,
 6,
 8,
 8,
 9,
 9,
 8,
 9,
 9,
 8,
 10,
 8,
 7,
 7,
 5,
 12,
 8,
 8,
 12,
 7,
 10,
 7,
 8,
 8,
 8,
 11,
 7,
 8,
 9,
 7,
 7,
 6,
 5,
 9,
 8,
 10,
 8,
 7,
 12,
 9,
 9,
 10,
 7,
 9,
 5,
 9,
 7,
 9,
 7,
 7,
 8,
 11,
 7,
 7,
 6,
 7,
 9,
 10,
 9,
 8,
 9,
 7,
 7,
 9,
 8,
 9,
 8,
 9,
 10,
 7,
 7,
 7,
 8,
 9,
 6,
 11,
 7,
 10,
 7,
 11,
 10,
 6,
 8,
 9,
 10,
 8,
 7,
 7,
 10,
 7,
 12,
 9,
 11,
 8,
 9,
 8,
 9,
 6,
 7,
 7,
 9,
 8,
 8,
 7,
 9,
 8,
 7,
 8,
 11,
 11,
 9,
 8,
 11,
 10,
 7,
 8,
 7,
 10,
 8,
 9,
 11,
 11,
 9,
 7,
 8,
 9,
 10,
 8,
 8,
 8,
 9,
 9,
 9,
 6,
 7,
 8,
 13,
 12,
 8,
 7,
 7,
 10,
 12,
 13,
 10,
 6,
 13,
 10,
 8,
 9,
 6,
 12,
 7,
 9,
 

In [12]:
import numpy as np
max_length = int(np.percentile(max_length, 95))

In [13]:
#vectorization
text_vector = tf.keras.layers.TextVectorization(max_tokens=vocab_length,
                                                output_sequence_length=max_length)

In [14]:
text_vector.adapt(df.String)

In [15]:
text_vector.get_vocabulary()[:5]

['', '[UNK]', 'for', 'with', 'and']

In [16]:
text_vector(["Luxurious ingredient for gourmet dining"])

<tf.Tensor: shape=(1, 12), dtype=int64, numpy=
array([[ 199, 1126,    2,  151,  382,    0,    0,    0,    0,    0,    0,
           0]])>

In [17]:
# embedding
embedding = tf.keras.layers.Embedding(input_dim = vocab_length,
                                      input_length=max_length,
                                      output_dim=1024,
                                      mask_zero=True)

In [18]:
embed = embedding(text_vector(["Luxurious ingredient for gourmet dining"]))

In [19]:
embed

<tf.Tensor: shape=(1, 12, 1024), dtype=float32, numpy=
array([[[ 0.04043684,  0.04742959, -0.02143621, ...,  0.04339835,
         -0.00970022,  0.04381562],
        [-0.00509573,  0.00864817,  0.02083527, ...,  0.01732795,
         -0.04011247, -0.01027735],
        [ 0.04611878,  0.01603084, -0.02062531, ..., -0.04476657,
          0.04253707,  0.0163291 ],
        ...,
        [ 0.01093497, -0.02421271,  0.04809925, ..., -0.04501803,
         -0.0125985 , -0.03588951],
        [ 0.01093497, -0.02421271,  0.04809925, ..., -0.04501803,
         -0.0125985 , -0.03588951],
        [ 0.01093497, -0.02421271,  0.04809925, ..., -0.04501803,
         -0.0125985 , -0.03588951]]], dtype=float32)>

In [20]:
one_hot_encoded_train = tf.one_hot(y_train, depth=101)
one_hot_encoded_test = tf.one_hot(y_test, depth=101)

In [21]:
train_data_one_hot = tf.data.Dataset.from_tensor_slices((X_train, one_hot_encoded_train)).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
test_data_one_hot = tf.data.Dataset.from_tensor_slices((X_test, one_hot_encoded_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [33]:
# conv1d
inputs = tf.keras.layers.Input(shape=(1, ), dtype='string')
x = text_vector(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True))(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(x)
# x = tf.keras.layers.Conv1D(256, kernel_size=3)(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.GRU(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_list), activation='softmax')(x)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch : 3e-3/(1+epoch)**(2))

model = tf.keras.models.Model(inputs, outputs)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 12)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 12, 1024)          25600000  
                                                                 
 bidirectional_1 (Bidirecti  (None, 12, 1024)          4724736   
 onal)                                                           
                                                                 
 gru_1 (GRU)                 (None, 128)               443136    
                                                                 
 batch_normalization_4 (Bat  (None, 128)               512 

In [34]:
history = model.fit(train_data_one_hot,
                    epochs=10,
                    validation_data=(test_data_one_hot),
                    validation_steps=int(0.25*len(test_data_one_hot)),
                    steps_per_epoch=len(train_data_one_hot),
                    callbacks=lr_scheduler)

Epoch 1/10
31/31 [==============================] - 12s 148ms/step - loss: 2.6294 - accuracy: 0.4808 - val_loss: 3.6414 - val_accuracy: 0.5547 - lr: 0.0030
Epoch 2/10
31/31 [==============================] - 2s 47ms/step - loss: 0.6037 - accuracy: 0.9017 - val_loss: 3.1365 - val_accuracy: 0.6719 - lr: 7.5000e-04
Epoch 3/10
31/31 [==============================] - 1s 31ms/step - loss: 0.3095 - accuracy: 0.9567 - val_loss: 2.8524 - val_accuracy: 0.6797 - lr: 3.3333e-04
Epoch 4/10
31/31 [==============================] - 1s 22ms/step - loss: 0.2451 - accuracy: 0.9753 - val_loss: 2.6154 - val_accuracy: 0.6797 - lr: 1.8750e-04
Epoch 5/10
31/31 [==============================] - 1s 31ms/step - loss: 0.2049 - accuracy: 0.9803 - val_loss: 2.3894 - val_accuracy: 0.6797 - lr: 1.2000e-04
Epoch 6/10
31/31 [==============================] - 1s 32ms/step - loss: 0.1884 - accuracy: 0.9814 - val_loss: 2.1879 - val_accuracy: 0.6875 - lr: 8.3333e-05
Epoch 7/10
31/31 [==============================] - 1s

In [35]:
model.evaluate(test_data_one_hot)

1/8 [==>...........................] - ETA: 0s - loss: 1.6710 - accuracy: 0.6875

8/8 [==============================] - 0s 14ms/step - loss: 1.5141 - accuracy: 0.7238


[1.5141034126281738, 0.7237903475761414]

In [41]:
model.save("75_3_gru_one_hot/")

INFO:tensorflow:Assets written to: 75_3_gru_one_hot/assets


INFO:tensorflow:Assets written to: 75_3_gru_one_hot/assets


In [28]:
tf.argmax(model.predict(["Cooking tutorials for achieving the perfect macaron feet"]), axis=1)

1/1 [==============================] - 2s 2s/step


<tf.Tensor: shape=(1,), dtype=int64, numpy=array([55])>

In [30]:
class_list[55]

'tuna_tartare'

# Pretrained embeddings

In [36]:
import tensorflow_hub as hub
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")

In [37]:
embed(["Rice cooked food"])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-0.02827955,  0.04139925,  0.0158745 ,  0.04136196,  0.05827901,
         0.03923946, -0.02325298,  0.00041435, -0.0523848 ,  0.05865195,
        -0.02272667, -0.06025485, -0.06244032, -0.04705139, -0.0854217 ,
         0.02652335, -0.01217343, -0.06420659,  0.00995875, -0.04074476,
         0.04196679,  0.07901078, -0.01970854, -0.01796671,  0.04481786,
         0.05687874, -0.04481113,  0.02043473, -0.07764187, -0.03506602,
         0.04480224,  0.06017666, -0.06590992, -0.00959308, -0.06850488,
         0.01169699, -0.01823763, -0.04149666,  0.04416808, -0.00070136,
         0.02090705, -0.01576048,  0.08526254, -0.06790678, -0.08336893,
         0.05019844, -0.00493785, -0.0217003 , -0.06885814, -0.08142836,
        -0.01130935,  0.05944003,  0.0343403 , -0.024447  ,  0.02204055,
         0.00451931, -0.0093224 ,  0.00847749,  0.07504914, -0.01754982,
        -0.01578058, -0.0539532 , -0.0403356 ,  0.03714855,  0.00210826,
 

In [38]:
sentence_encoder_layer = hub.KerasLayer(embed,
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=True)

In [43]:
model_1 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    # tf.keras.layers.Conv1D(64, kernel_size=2),
    # tf.keras.layers.GlobalAveragePooling1D,
    # tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(class_list), activation='softmax')
])
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy'])

In [44]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_10 (Dense)            (None, 101)               51813     
                                                                 
Total params: 256849637 (979.80 MB)
Trainable params: 51813 (202.39 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [45]:
history_1 = model_1.fit(train_data_one_hot,
                    epochs=5,
                    validation_data=(test_data_one_hot),
                    validation_steps=int(0.2*len(test_data_one_hot)),
                    steps_per_epoch=len(train_data_one_hot))

Epoch 1/5
31/31 [==============================] - 1s 15ms/step - loss: 4.5208 - accuracy: 0.1371 - val_loss: 4.3975 - val_accuracy: 0.2500
Epoch 2/5
31/31 [==============================] - 0s 6ms/step - loss: 4.2335 - accuracy: 0.5615 - val_loss: 4.1786 - val_accuracy: 0.3906
Epoch 3/5
31/31 [==============================] - 0s 6ms/step - loss: 3.9766 - accuracy: 0.6820 - val_loss: 3.9832 - val_accuracy: 0.4688
Epoch 4/5
31/31 [==============================] - 0s 7ms/step - loss: 3.7297 - accuracy: 0.7283 - val_loss: 3.7664 - val_accuracy: 0.4844
Epoch 5/5
31/31 [==============================] - 0s 7ms/step - loss: 3.4942 - accuracy: 0.7656 - val_loss: 3.5908 - val_accuracy: 0.5156


In [46]:
model_1.evaluate(test_data_one_hot)

1/8 [==>...........................] - ETA: 0s - loss: 3.5908 - accuracy: 0.5156

8/8 [==============================] - 0s 6ms/step - loss: 3.6407 - accuracy: 0.5746


[3.640739679336548, 0.5745967626571655]